### Notebook for inspecting stereo results

In [2]:
from itertools import cycle
from pathlib import Path
from typing import Iterator, NamedTuple

import cv2
import numpy as np

from mynd.image import ImageType, ImageComposite, ImageCompositeLoader
from mynd.tasks.common import build_image_composite_loaders
from mynd.visualization import (
    WindowHandle,
    KeyCode,
    create_image_visualizer,
    render_image,
    destroy_all_windows,
)

from mynd.utils.log import logger


class ImageDirectories(NamedTuple):
    """Class representing image directories."""

    colors: Path
    ranges: Path
    normals: Path


def normalize_values(values: np.ndarray) -> np.ndarray:
    """Normalizes an array."""
    min_value: float = values.min()
    max_value: float = values.max()

    normalized: np.ndarray = (values - min_value) / (
        max_value - min_value
    ) * -255 + 255
    return normalized.astype(np.uint8)


class InspectData(NamedTuple):

    label: str
    colors: np.ndarray
    ranges: np.ndarray


def on_render(
    window: WindowHandle, colors: np.ndarray, ranges: np.ndarray
) -> None:
    """Callback to render during inspection."""
    stacked: np.ndarray = np.hstack((colors, ranges))
    render_image(window, stacked)


def on_update(
    keys: Iterator[str], loaders: dict[str, ImageCompositeLoader]
) -> InspectData:
    """Callback to update during inspection."""
    key: str = next(keys)
    loader: ImageCompositeLoader = loaders.get(key)
    composite: ImageComposite = loader()

    colors: np.ndarray = composite.get(ImageType.COLOR).to_array()
    ranges: np.ndarray = composite.get(ImageType.RANGE).to_array()

    logger.info(
        f"Range statistics - min: {ranges.min()}, max: {ranges.max()}, median: {np.median(ranges)}"
    )

    # TODO: Filter far?
    FAR: float = 7.0
    ranges: np.ndarray = ranges

    # TODO: Add non-linear scaling
    ranges: np.ndarray = normalize_values(ranges)
    ranges = cv2.applyColorMap(ranges, cv2.COLORMAP_JET)

    # Convert colors to uint8 - NOTE converts 3D arrays to 2D
    # colors: np.ndarray = cv2.convertScaleAbs(colors, alpha=(255.0/65535.0))
    colors: np.ndarray = np.squeeze(colors)

    if colors.ndim == 2:
        colors: np.ndarray = np.stack([colors] * 3, axis=2)

    logger.info(colors.shape)

    # TODO: Add statistics
    return InspectData(key, colors, ranges)


def inspect_stereo_geometry(loaders: dict[str, ImageCompositeLoader]) -> None:
    """Inspect stereo geometry by loading images and their correspond range map."""

    keys: Iterator[str] = cycle(sorted(list(loaders.keys())))
    data: InspectData = on_update(keys, loaders)

    assert data.colors is not None, "color map does not exist"
    assert data.ranges is not None, "range map does not exist"

    window: WindowHandle = create_image_visualizer(
        window_name="color", width=1360, height=512
    )

    while True:

        # On stereo inspect update
        on_render(window, data.colors, data.ranges)
        key: KeyCode = KeyCode(cv2.waitKey(0))

        match key:
            case KeyCode.SPACE:
                data: InspectData = on_update(keys, loaders)
            case KeyCode.ESC:
                destroy_all_windows()
                logger.info("Quitting...")
                return
            case _:
                pass


def main() -> None:
    """Main function."""

    # r23m7ms0_20100606_001908
    # r23685bc_20100605_021022, r23685bc_20120530_233021, r23685bc_20140616_225022
    DEPLOYMENT: str = "r23m7ms0_20100606_001908"

    image_directories: dict[ImageType, Path] = {
        ImageType.COLOR: Path(
            f"/data/kingston_snv_01/acfr_images_grayworld/{DEPLOYMENT}_grayworld"
        ),
        ImageType.RANGE: Path(
            f"/data/kingston_snv_01/acfr_stereo_ranges/{DEPLOYMENT}_ranges"
        ),
        ImageType.NORMAL: Path(
            f"/data/kingston_snv_01/acfr_stereo_normals/{DEPLOYMENT}_normals"
        ),
    }

    for image_type, directory in image_directories.items():
        assert directory.exists(), f"directory does not exist: {directory}"

    loaders: dict[str, ImageCompositeLoader] = build_image_composite_loaders(
        image_directories
    )

    inspect_stereo_geometry(loaders)


# INVOKE MAIN
main()

2024-10-20 11:53:30.713 | INFO     | __main__:on_update:60 - Range statistics - min: 0.0, max: 4.907574653625488, median: 3.2675328254699707
2024-10-20 11:53:30.718 | INFO     | __main__:on_update:77 - (1024, 1360, 3)
2024-10-20 11:53:33.799 | INFO     | __main__:on_update:60 - Range statistics - min: 0.0, max: 4.801724433898926, median: 3.268348217010498
2024-10-20 11:53:33.804 | INFO     | __main__:on_update:77 - (1024, 1360, 3)
2024-10-20 11:53:39.458 | INFO     | __main__:on_update:60 - Range statistics - min: 0.0, max: 4.7857818603515625, median: 3.2544150352478027
2024-10-20 11:53:39.461 | INFO     | __main__:on_update:77 - (1024, 1360, 3)
2024-10-20 11:53:40.343 | INFO     | __main__:on_update:60 - Range statistics - min: 0.0, max: 4.773796081542969, median: 3.228394031524658
2024-10-20 11:53:40.346 | INFO     | __main__:on_update:77 - (1024, 1360, 3)
2024-10-20 11:53:43.319 | INFO     | __main__:on_update:60 - Range statistics - min: 0.0, max: 4.30046272277832, median: 3.129526